In [1]:
# ============================================================
# CHALLENGE 2: p-factor regression baseline using braindecode models
# ============================================================

import os, glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import mne
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# -------------------------
# Configuration
# -------------------------
BIDS_ROOT = "/data5/open_data/HBN/EEG_BIDS"
DATASETS = [f"ds0055{i:02d}" for i in range(5, 17)]  # ds005505~ds005516
TASKS = ["RestingState",
         "DespicableMe",
         "FunWithFractals",
         "ThePresent",
         "DiaryOfAWimpyKid",
         "contrastChangeDetection",
         "surroundSupp",
         "seqLearning6target",
         "seqLearning8target",
         "symbolSearch",]

TARGET_SFREQ = 100
N_CHANS = 128
WIN_SIZE = 1000      # 10초 @100Hz
STRIDE_SIZE = 500    # 5초 overlap

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


print(f"Device: {DEVICE}")

# 캐시 저장 경로
CACHE_DIR = "./cache_eeg_windows"
os.makedirs(CACHE_DIR, exist_ok=True)


Device: cuda


In [10]:
# ------------------------------------------------------------
# participants.tsv 병합
# ------------------------------------------------------------
def load_all_participants(bids_root=BIDS_ROOT, datasets=DATASETS):
    dfs = []
    for ds in datasets:
        pfile = os.path.join(bids_root, ds, "participants.tsv")
        if os.path.exists(pfile):
            df = pd.read_csv(pfile, sep="\t")
            df["release"] = ds
            dfs.append(df)
    df_all = pd.concat(dfs, ignore_index=True)
    df_all = df_all.drop_duplicates(subset=["participant_id"])
    df_all = df_all.set_index("participant_id")
    return df_all


# ------------------------------------------------------------
# EEG 파일 목록 수집
# ------------------------------------------------------------
def collect_eeg_files(bids_root=BIDS_ROOT, datasets=DATASETS, tasks=TASKS):
    all_files = []
    for ds in datasets:
        for task in tasks:
            bids_path = os.path.join(bids_root, ds)
            eeg_pattern1 = os.path.join(bids_path, "sub-*", "eeg", f"sub-*_task-{task}_eeg.set")
            eeg_pattern2 = os.path.join(bids_path, "sub-*", "eeg", f"sub-*_task-{task}_run-*_eeg.set")
            eeg_files = glob.glob(eeg_pattern1) + glob.glob(eeg_pattern2)
            for f in eeg_files:
                sub_id = os.path.basename(f).split("_")[0]  # e.g., sub-0001
                all_files.append({"release": ds, "participant_id": sub_id, "task": task, "path": f})
    return pd.DataFrame(all_files)


# ------------------------------------------------------------
# MNE 전처리 + 고정 윈도우 생성
# ------------------------------------------------------------
from braindecode.preprocessing import preprocess, Preprocessor
from braindecode.datautil import create_fixed_length_windows
from braindecode.datasets import BaseConcatDataset, BaseDataset
import gc



TARGET_SFREQ = 100
N_CHANS = 128
WIN_SIZE = 1000      # 10초 @100Hz
STRIDE_SIZE = 500    # 5초 overlap


def load_and_preprocess(file_path, is_val=False, sfreq=TARGET_SFREQ):
    base_name = os.path.basename(file_path).replace(".set", "")
    cache_name = f"{base_name}_{'val' if is_val else 'train'}_{sfreq}Hz.npy"
    cache_path = os.path.join(CACHE_DIR, cache_name)

    # 이미 캐시 존재 시 스킵
    if os.path.exists(cache_path):
        return "cached"

    try:
        raw = mne.io.read_raw_eeglab(file_path, preload=True, verbose="ERROR")
    except Exception as e:
        return f"[EEG load error] {os.path.basename(file_path)}: {e}"

    rec_dur = raw.n_times / raw.info["sfreq"]
    if rec_dur < 900:
        del raw; gc.collect()
        return f"[Skip short file] {os.path.basename(file_path)} ({rec_dur/60:.1f} min)"

    try:
        base_ds = BaseDataset(raw, description={"path": file_path})
        concat_ds = BaseConcatDataset([base_ds])
        preprocess(concat_ds, [
            Preprocessor("pick_types", eeg=True, stim=False),
            Preprocessor("filter", l_freq=0.5, h_freq=50.0),
            Preprocessor(lambda data: data * 1e6),
            Preprocessor("resample", sfreq=sfreq)
        ])
        windows_ds = create_fixed_length_windows(
            concat_ds, window_size_samples=WIN_SIZE,
            window_stride_samples=STRIDE_SIZE, drop_last_window=True
        )

        X = np.array([
            w[0].get_data() if hasattr(w[0], "get_data") else np.array(w[0])
            for w in windows_ds
        ], dtype=np.float32)

        np.save(cache_path, X)
        os.sync()
        del raw, base_ds, concat_ds, windows_ds
        gc.collect()
        return f"[Saved] {cache_name}"

    except Exception as e:
        del raw; gc.collect()
        return f"[Preprocess failed] {os.path.basename(file_path)}: {e}"


/home/RA/.local/lib/python3.10/site-packages/braindecode/datautil/__init__.py:43: UserWarning: create_fixed_length_windows has been moved to preprocessing, please use from braindecode.preprocessing import create_fixed_length_windows
  warn(


In [9]:
import sys
from tqdm import tqdm

def build_cache(files_df, is_val=False):
    rows = list(files_df.itertuples(index=False))
    skip_logs = []  # 스킵/에러 로그 저장 (아래쪽에 누적 표시용)
    
    pbar = tqdm(rows, desc=f"Build cache ({'val' if is_val else 'train'})", ncols=90, leave=True)

    for r in pbar:
        result = load_and_preprocess(r.path, is_val=is_val)

        # 캐시 생성 결과 로그 수집
        if isinstance(result, str) and result not in ("ok", "cached"):
            skip_logs.append(result)

        # tqdm 진행바는 stderr로, 로그는 stdout으로 출력
        # 매 5개마다 최신 로그를 아래쪽에 덮어쓰기 방식으로 출력
        if len(skip_logs) % 5 == 0:
            sys.stdout.write("\033[2K\r")   # 현재 줄 지우기
            sys.stdout.write("\n".join(skip_logs[-5:]) + "\n")  # 최근 5개 로그만 표시
            sys.stdout.flush()

    pbar.close()

    # 완료 후 전체 로그 한 번 더 출력
    sys.stdout.write("\n=== Summary of skipped/failed files ===\n")
    sys.stdout.write("\n".join(skip_logs[-20:]) + "\n")
    sys.stdout.flush()


In [4]:
class EEGPfactorDataset(Dataset):
    def __init__(self, files_df, labels_df, is_val=False):
        self.samples = []
        for _, row in tqdm(files_df.iterrows(), total=len(files_df)):
            sub_id = row["participant_id"]
            if sub_id not in labels_df.index:
                continue
            y = torch.tensor(labels_df.loc[sub_id, "p_factor"], dtype=torch.float32)

            X_all = load_and_preprocess(row["path"], is_val=is_val)
            if X_all is None:
                continue

            # Subject 단위 평균 feature (C, T)
            X_mean = torch.tensor(X_all.mean(axis=0), dtype=torch.float32)
            self.samples.append((X_mean, y))

    def __len__(self): return len(self.samples)
    def __getitem__(self, idx): return self.samples[idx]


In [5]:
from braindecode.models import EEGNetv4, ShallowFBCSPNet, EEGConformer

def get_model(model_name, n_chans=N_CHANS, n_times=1000):
    if model_name == "EEGNet":
        m = EEGNetv4(n_chans, n_classes=1, input_window_samples=n_times, final_conv_length="auto")
        m.classifier[-1] = nn.Linear(m.classifier[-1].in_features, 1)
    elif model_name == "ShallowFBCSPNet":
        m = ShallowFBCSPNet(n_chans, n_classes=1, input_window_samples=n_times, final_conv_length=30)
        m.regressor = nn.Linear(m.classifier.in_features, 1)
    elif model_name == "EEGConformer":
        m = EEGConformer(n_chans=n_chans, n_outputs=1, input_window_samples=n_times)
    # elif model_name == "EEGInception":
    #     m = EEGInception(n_chans, n_outputs=1, input_window_samples=n_times)
    else:
        raise ValueError(f"Unknown model: {model_name}")
    return m


In [6]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tqdm import tqdm

def train_model(model, train_loader, val_loader, model_name, n_epochs=30, lr=1e-4):
    model = model.to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-3)

    best_mae, best_mse, best_r2 = float("inf"), float("inf"), -1.0

    for ep in range(1, n_epochs + 1):
        # ============================================================
        # ① Training
        # ============================================================
        model.train()
        train_mae, train_mse = 0.0, 0.0

        pbar = tqdm(train_loader, desc=f"[{model_name}] Epoch {ep}/{n_epochs} (train)", leave=False)
        for X, y in pbar:
            X, y = X.to(DEVICE), y.to(DEVICE)
            pred = model(X).squeeze()

            loss_mae = F.l1_loss(pred, y)
            loss_mse = F.mse_loss(pred, y)
            loss = 0.5 * loss_mae + 0.5 * loss_mse

            opt.zero_grad()
            loss.backward()
            opt.step()

            train_mae += loss_mae.item()
            train_mse += loss_mse.item()
            pbar.set_postfix({"L1": f"{loss_mae.item():.4f}", "L2": f"{loss_mse.item():.4f}"})

        avg_mae = train_mae / len(train_loader)
        avg_mse = train_mse / len(train_loader)

        # ============================================================
        # ② Validation
        # ============================================================
        model.eval()
        preds, trues = [], []
        pbar_val = tqdm(val_loader, desc=f"[{model_name}] Epoch {ep}/{n_epochs} (val)", leave=False)
        with torch.no_grad():
            for X, y in pbar_val:
                X, y = X.to(DEVICE), y.to(DEVICE)
                p = model(X).squeeze()
                preds.extend(p.cpu().numpy())
                trues.extend(y.cpu().numpy())

        preds, trues = np.array(preds), np.array(trues)
        val_mae = mean_absolute_error(trues, preds)
        val_mse = mean_squared_error(trues, preds)
        val_r2  = r2_score(trues, preds)

        print(f"[{model_name}] Epoch {ep:02d} | "
              f"Train MAE={avg_mae:.4f}, MSE={avg_mse:.4f} | "
              f"Val MAE={val_mae:.4f}, MSE={val_mse:.4f}, R²={val_r2:.3f}")

        # ============================================================
        # ③ Checkpoint
        # ============================================================
        if val_mae < best_mae:
            best_mae, best_mse, best_r2 = val_mae, val_mse, val_r2
            torch.save(model.state_dict(), f"best_{model_name}.pth")

    print(f"\n[{model_name}] Best Val -> MAE={best_mae:.4f}, MSE={best_mse:.4f}, R²={best_r2:.3f}\n")
    return best_mae, best_mse, best_r2


In [7]:
# ------------------------------------------------------------
# 메타 / 파일 리스트 불러오기
# ------------------------------------------------------------
meta_df = load_all_participants()          # participants.tsv 전체 병합
files_df = collect_eeg_files()             # EEG 파일 경로 수집
labels_df = meta_df[["p_factor"]].dropna() # p-factor 라벨 포함 subject만 사용

# ------------------------------------------------------------
# Train/Val 분리 (release 기준)
# ------------------------------------------------------------
train_releases = [f"ds0055{i:02d}" for i in range(5, 17) if i != 10 and i != 5]  ######
val_releases   = ["ds005509"]  

train_files = files_df[files_df["release"].isin(train_releases)]
val_files   = files_df[files_df["release"].isin(val_releases)]

train_files = train_files[train_files["participant_id"].isin(labels_df.index)]
val_files   = val_files[val_files["participant_id"].isin(labels_df.index)]

print(f"Train files: {len(train_files)}, Val files: {len(val_files)}")

Train files: 21393, Val files: 2966


In [13]:
train_releases = ["ds005512", "ds005514", "ds005515", "ds005516"]
train_files = files_df[files_df["release"].isin(train_releases)]
train_files = train_files[train_files["participant_id"].isin(labels_df.index)]
build_cache(train_files, is_val=False)

Build cache (train):   0%|                             | 4/9794 [00:04<2:57:54,  1.09s/it]

[Skip short file] sub-NDARAA947ZG5_task-RestingState_eeg.set (11.1 min)
[Skip short file] sub-NDARAB514MAJ_task-RestingState_eeg.set (5.3 min)
[Skip short file] sub-NDARAB977GFB_task-RestingState_eeg.set (6.2 min)
[Skip short file] sub-NDARAD571FLB_task-RestingState_eeg.set (7.3 min)
[Skip short file] sub-NDARAF003VCL_task-RestingState_eeg.set (5.8 min)


Build cache (train):   0%|                             | 9/9794 [00:10<3:01:14,  1.11s/it]

[Skip short file] sub-NDARAG191AE8_task-RestingState_eeg.set (12.7 min)
[Skip short file] sub-NDARAG610XL4_task-RestingState_eeg.set (6.3 min)
[Skip short file] sub-NDARAH976BND_task-RestingState_eeg.set (8.1 min)
[Skip short file] sub-NDARAJ977PRJ_task-RestingState_eeg.set (5.9 min)
[Skip short file] sub-NDARAN934AZL_task-RestingState_eeg.set (5.9 min)


Build cache (train):   0%|                            | 14/9794 [00:15<2:45:42,  1.02s/it]

...

[Skip short file] sub-NDARZA863HD3_task-symbolSearch_eeg.set (2.9 min)
[Skip short file] sub-NDARZA879DJG_task-symbolSearch_eeg.set (4.9 min)
[Skip short file] sub-NDARZC423JYT_task-symbolSearch_eeg.set (3.4 min)
[Skip short file] sub-NDARZE459MJL_task-symbolSearch_eeg.set (3.4 min)
[Skip short file] sub-NDARZG044CJ5_task-symbolSearch_eeg.set (3.5 min)


Build cache (train): 100%|█████████████████████████▉| 9791/9794 [4:37:16<00:04,  1.35s/it]

[Skip short file] sub-NDARZG265XMR_task-symbolSearch_eeg.set (2.7 min)
[Skip short file] sub-NDARZR075BR2_task-symbolSearch_eeg.set (2.9 min)
[Skip short file] sub-NDARZR463DUN_task-symbolSearch_eeg.set (3.6 min)
[Skip short file] sub-NDARZR700EG6_task-symbolSearch_eeg.set (7.1 min)
[Skip short file] sub-NDARZT876LP6_task-symbolSearch_eeg.set (5.1 min)


Build cache (train): 100%|██████████████████████████| 9794/9794 [4:37:19<00:00,  1.70s/it]


=== Summary of skipped/failed files ===
[Skip short file] sub-NDARYD717TM7_task-symbolSearch_eeg.set (3.0 min)
[Skip short file] sub-NDARYH224RXX_task-symbolSearch_eeg.set (2.7 min)
[Skip short file] sub-NDARYH501UH3_task-symbolSearch_eeg.set (2.6 min)
[Skip short file] sub-NDARYJ382ZE3_task-symbolSearch_eeg.set (2.7 min)
[Skip short file] sub-NDARYJ413BLN_task-symbolSearch_eeg.set (5.5 min)
[Skip short file] sub-NDARYL835ZVX_task-symbolSearch_eeg.set (4.1 min)
[Skip short file] sub-NDARYT509KUF_task-symbolSearch_eeg.set (3.4 min)
[Skip short file] sub-NDARZA115BW6_task-symbolSearch_eeg.set (2.6 min)
[Skip short file] sub-NDARZA863HD3_task-symbolSearch_eeg.set (2.9 min)
[Skip short file] sub-NDARZA879DJG_task-symbolSearch_eeg.set (4.9 min)
[Skip short file] sub-NDARZC423JYT_task-symbolSearch_eeg.set (3.4 min)
[Skip short file] sub-NDARZE459MJL_task-symbolSearch_eeg.set (3.4 min)
[Skip short file] sub-NDARZG044CJ5_task-symbolSearch_eeg.set (3.5 min)
[Skip short file] sub-NDARZG265XMR_t

In [11]:
print("\n=== Building Train Cache ===")
build_cache(train_files, is_val=False)
print("\n=== Building Val Cache ===")
build_cache(val_files, is_val=True)


=== Building Train Cache ===


Build cache (train):   0%|                           | 4/21393 [00:44<65:44:38, 11.07s/it]

[Skip short file] sub-NDARAB793GL3_task-RestingState_eeg.set (6.8 min)
[Skip short file] sub-NDARAH948UF0_task-RestingState_eeg.set (5.7 min)
[Skip short file] sub-NDARAM675UR8_task-RestingState_eeg.set (6.0 min)
[Skip short file] sub-NDARAP176AD1_task-RestingState_eeg.set (6.0 min)
[Skip short file] sub-NDARAV031PPJ_task-RestingState_eeg.set (7.2 min)


Build cache (train):   0%|                           | 6/21393 [01:06<64:05:07, 10.79s/it]

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3301 samples (6.602 s)



...

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/home/RA/.local/lib/python3.10/site-packages/braindecode/preprocessing/preprocess.py:71: UserWarning: Preprocessing choices with lambda functions cannot be saved.
  warn("Preprocessing choices with lambda functions cannot be saved.")


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3301 samples (6.602 s)



Build cache (train):  59%|███████████▊        | 12652/21393 [42:14:48<29:11:14, 12.02s/it]


KeyboardInterrupt: 

In [12]:
train_ds = EEGPfactorDataset(train_files, labels_df)
val_ds   = EEGPfactorDataset(val_files, labels_df)

  0%|          | 0/21393 [00:09<?, ?it/s]

[Skip short file] /data5/open_data/HBN/EEG_BIDS/ds005506/sub-NDARAB793GL3/eeg/sub-NDARAB793GL3_task-RestingState_eeg.set (6.8 min)


  0%|          | 1/21393 [00:18<79:49:22, 13.43s/it]

[Skip short file] /data5/open_data/HBN/EEG_BIDS/ds005506/sub-NDARAH948UF0/eeg/sub-NDARAH948UF0_task-RestingState_eeg.set (5.7 min)


  0%|          | 2/21393 [00:29<63:57:58, 10.77s/it]

...

  0%|          | 9/21393 [01:40<61:39:14, 10.38s/it]

[Skip short file] /data5/open_data/HBN/EEG_BIDS/ds005506/sub-NDARBU607ZZ3/eeg/sub-NDARBU607ZZ3_task-RestingState_eeg.set (6.0 min)


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f9151c8c5e0>>
Traceback (most recent call last):
  File "/home/RA/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
  0%|          | 10/21393 [01:49<52:44:05,  8.88s/it]

[Skip short file] /data5/open_data/HBN/EEG_BIDS/ds005506/sub-NDARBU730PN8/eeg/sub-NDARBU730PN8_task-RestingState_eeg.set (5.9 min)


  0%|          | 11/21393 [01:51<52:13:51,  8.79s/it]

In [ ]:
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=4, pin_memory=True, prePatch_factor=2)
val_loader   = DataLoader(val_ds, batch_size=8, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
# 모델 비교 학습
models = ["EEGNet", "ShallowFBCSPNet", "EEGConformer"]
results = {}
for name in models:
    model = get_model(name)
    mae, r2 = train_model(model, train_loader, val_loader, name)
    results[name] = {"MAE": mae, "R²": r2}

print("\n=== Model Comparison ===")
for m, v in results.items():
    print(f"{m:20s} | MAE={v['MAE']:.4f} | R²={v['R²']:.4f}")


In [ ]:
def visualize_representation(model, data_loader, title="t-SNE of EEG representations"):
    model.eval(); feats, labels = [], []
    with torch.no_grad():
        for X, y in data_loader:
            X = X.to(DEVICE)
            # feature 추출 (EEGNet 기준으로 feature extractor 부분 접근)
            h = model.classifier[0](X) if hasattr(model, "classifier") else model(X)
            h = h.mean(dim=-1).view(X.size(0), -1).cpu().numpy()
            feats.append(h); labels.append(y.numpy())
    feats = np.concatenate(feats); labels = np.concatenate(labels)

    Z = TSNE(n_components=2, perplexity=30).fit_transform(feats)
    plt.figure(figsize=(7,6))
    sc = plt.scatter(Z[:,0], Z[:,1], c=labels, cmap='viridis', alpha=0.7)
    plt.colorbar(sc, label="p-factor")
    plt.title(title)
    plt.show()

# 예시 시각화
best_model = get_model("EEGNet")
best_model.load_state_dict(torch.load("best_EEGNet.pth", map_location=DEVICE))
best_model.to(DEVICE)
visualize_representation(best_model, val_loader)
